In [ ]:
import numpy as np
import pandas as pd

from sklearn import model_selection, preprocessing

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, SGDClassifier
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

In [ ]:
# Récuperer les données
df = pd.read_csv('./Data/Donnees_Quotidiennes.csv', parse_dates=['Date'],sep=';')

In [ ]:
# suppression des champs inutiles.
df  = df[['Code INSEE région','Région','Date','Consommation (MW)', 'Température (°C)']]
# Supprimer les lignes avec des valeurs absents
df.dropna(axis = 0, subset=["Consommation (MW)"], inplace=True)

In [ ]:
#Application d'un filtre sur la région pour ne travailler pour le moment que sur la région Ile de france afin de simplifier les tests.
df_idf = df[df['Code INSEE région']== 11]


In [ ]:
# suppression des colonnes devenus inutiles
df_idf.drop(['Code INSEE région', 'Région'],inplace=True,axis=1)

In [ ]:
# Ajout de la colonne Jour
df_idf["Jour"] = pd.to_datetime(df_idf.Date).dt.weekday
df_idf["Mois"] = pd.to_datetime(df_idf.Date).dt.month
df_idf["Annee"] = pd.to_datetime(df_idf.Date).dt.year


In [ ]:
# Dichotomisation de la colonne Jour, Mois, Annee
df_idf = df_idf.join(pd.get_dummies(df_idf.Jour, prefix='Jour'))
df_idf = df_idf.join(pd.get_dummies(df_idf.Mois, prefix='Mois'))
df_idf = df_idf.join(pd.get_dummies(df_idf.Annee, prefix='Annee'))

In [ ]:
# paramétrage de la date en tant qu'index.
df_idf.set_index('Date',inplace=True)

In [ ]:
df_idf = df_idf.reset_index()

In [ ]:
# normalisation des colonnes ['Consommation (MW)', 'Température (°C)']
scaler = preprocessing.StandardScaler().fit(df_idf[['Consommation (MW)', 'Température (°C)']])

df_idf[['Consommation (MW)', 'Température (°C)']] = pd.DataFrame(scaler.transform(df_idf[['Consommation (MW)', 'Température (°C)']]), index= df_idf.index)


In [ ]:
# Définition des variables cibles et du target
target = df_idf["Consommation (MW)"]
data = df_idf.drop(['Consommation (MW)', 'Jour', 'Mois', 'Annee'],axis=1)

In [ ]:
# Spliter les données
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.01, random_state=99, shuffle = False)


In [ ]:
# modèle de régression
from sklearn.linear_model import SGDRegressor
clf = SGDRegressor(penalty='elasticnet', l1_ratio=0.85)
clf.fit(X_train, y_train)

In [ ]:
clf.get_params()

In [ ]:
# Afficher l'intercept et les coefficients estimés pour chaque variable de data
coeffs = list(clf.coef_)
coeffs.insert(0, clf.intercept_)
feats = list(data.columns)
feats.insert(0, 'intercept')

values = pd.DataFrame({'valeur estimée': coeffs}, index = feats)
values.head()

In [ ]:
# Afficher les racines des erreurs quadratiques moyennes pour les deux échantillons, train et test
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(np.sqrt(mean_squared_error(y_train, pred_train)))
print(np.sqrt(mean_squared_error(y_test, pred_test)))

In [ ]:
# score R2
print("score train:",clf.score(X_train, y_train))
print("score test:", clf.score(X_test, y_test))

In [ ]:
moyenne = scaler.mean_[0]
ecart = scaler.scale_[0]
print("moyenne :", moyenne)
print("ecart-type", ecart)

In [ ]:
matrice = pd.DataFrame({'points_obsérvés': np.round((y_test*ecart)+moyenne), 
            'points_predits' : np.round((pred_test*ecart)+moyenne), 
            'T' : (X_test['Température (°C)']*scaler.scale_[-1])+scaler.mean_[-1]}, index = X_test.index)
matrice.head()

In [ ]:
# Graphique
font1 = {'family':'Times','color':'darkblue','size':18}
font2 = {'family':'Times','color':'black','size':14}

plt.plot(X_test.index, matrice['points_obsérvés'], label='Valeurs réellees', color='blue', marker='o',linestyle='-.')
plt.plot(X_test.index, matrice['points_predits'],label="Valeurs predites", color='red', marker='o',linestyle='-.')
plt.legend(labelcolor='markerfacecolor')
plt.title("Prédiction avec le modèle SGDRegressor", fontdict = font1)
plt.xticks(rotation=45)
plt.xlabel("Date", fontdict = font2)
plt.ylabel("Consommation Moyenne", fontdict = font2)
plt.show();



In [ ]:
# Graphique
font1 = {'family':'Times','color':'darkblue','size':18}
font2 = {'family':'Times','color':'black','size':14}


fig = plt.figure(figsize=(10,6))

ax1 = fig.add_subplot(111)
fig.autofmt_xdate(rotation=25)
ax1.set_title("Prédiction avec le modèle SGDRegressor Quotidien", fontdict = font1)
lns1 = ax1.plot(X_test.index, matrice['points_obsérvés'], label='Valeurs réellees', color='blue', marker='o',linestyle='-.')
lns2 = ax1.plot(X_test.index, matrice['points_predits'],label="Valeurs predites", color='red', marker='o',linestyle='-.')
ax1.set_ylabel("Consommation Moyenne", fontdict = font2)
ax1.set_xlabel("Date", fontdict = font2)

ax2 = ax1.twinx()
lns3 = ax2.plot(X_test.index, X_test['Température (°C)'], label='($^\circ$C)', color='grey', marker='o',linestyle='-.')
ax2.set_ylabel("Température Moyenne ($^\circ$C)", fontdict = font2)

# pour afficher les labels
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, labelcolor='markerfacecolor', loc='upper center')



In [ ]:
from math import pi
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import Legend, DatetimeTickFormatter, formatters, HoverTool, LinearAxis, Range1d

output_notebook()

In [ ]:
# List de tools
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset"

p = figure(plot_width = 600, plot_height = 400,     
           title = "Prédiction avec le modèle SGDRegressor Quotidien",                    
           x_axis_label = 'Date', x_axis_type="datetime",
           y_axis_label = 'Consommation Moyenne',
           tools=TOOLS)  


p.title.text_color = "darkblue"
p.title.text_font = "times"
p.title.text_font_size = "20px"
p.title.align = 'center'


p.line(X_test.index, matrice['points_obsérvés'], color = "navy", legend_label = "Valeurs réellees")   
p.circle(X_test.index, matrice['points_obsérvés'], color = "navy",fill_color='white', size=8)

p.line(X_test.index, matrice['points_predits'], color = "red", legend_label = "Valeurs predites") 
p.circle(X_test.index, matrice['points_predits'], color = "red", fill_color='white',size=8)

p.xaxis.major_label_orientation = pi/4
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.ticker.desired_num_ticks = 29


# Activation de l'interaction avec la légende
p.legend.location = "top_center"
p.legend.click_policy = 'hide'

# Style hover
p.add_tools(HoverTool(
    tooltips=[('Date', '@x{%Y-%m-%d}'),
        ('Consommation', '@y{0.00}')],
    formatters={'@x': 'datetime'}
))

show(p);


In [ ]:
#Source
source = ColumnDataSource(matrice)


# List de tools
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset"


y_overlimit = 0.05 
p = figure(plot_width = 600, plot_height = 400,     
           title = "Prédiction avec le modèle SGDRegressor Quotidien",                    
           x_axis_label = 'Date', x_axis_type="datetime",
           y_axis_label = 'Consommation Moyenne',
           toolbar_location="below",
           tools=TOOLS)  


p.title.text_color = "darkblue"
p.title.text_font = "times"
p.title.text_font_size = "20px"
p.title.align = 'center'


p.line(x='Date', y = 'points_obsérvés', color = "navy", legend_label = "Valeurs réellees", source = source)   
p.circle(x='Date', y ='points_obsérvés', color = "navy",fill_color='white', size=8, source = source)

p.line(x='Date', y ='points_predits', color = "red", legend_label = "Valeurs predites", source = source) 
p.circle(x='Date', y='points_predits', color = "red", fill_color='white',size=8, source = source)

# axis y, gauche
p.y_range = Range1d(matrice['points_obsérvés'].min() * (1 - y_overlimit), matrice['points_obsérvés'].max() * (1 + y_overlimit))

p.xaxis.major_label_orientation = pi/4
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.ticker.desired_num_ticks = 29

# Axis y, droite
y_column2_range = "T" + "_range"
p.extra_y_ranges = {
    y_column2_range: Range1d(
        start=matrice['T'].min() * (1 - y_overlimit),
        end=matrice['T'].max() * (1 + y_overlimit),
    )
}
p.add_layout(LinearAxis(y_range_name=y_column2_range), "right")

p.line( x='Date', y = 'T', color="grey", legend_label="T (C°)", y_range_name=y_column2_range, source = source)
p.circle(x='Date', y = 'T', color = "grey",fill_color='white', size=8, y_range_name=y_column2_range, source = source)


# Activation de l'interaction avec la légende
p.legend.location = "top_center"
p.legend.click_policy = 'hide'

# Style hover
p.add_tools(HoverTool(
    tooltips=[('Date', '@Date{%Y-%m-%d}'),
        ('Prédiction', '@{points_predits}{0.00}'),
        ('Valeur réelle', '@{points_obsérvés}{0.00}'),
        ('C°', "@T{0.00}")],
    formatters={'@Date': 'datetime'}
))

show(p);


In [ ]:
# Racine carrée de la Moyenne des résidus au carré.
# RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse(matrice['points_predits'], matrice['points_obsérvés'])

plus l'erreur quadratique moyenne est proche de 0, plus précises sont les prédictions.

Nous pourrons donc comparer ce résultat avec le RMSE des autres modèles

In [ ]:
# Calculer et afficher le score MAPE
y_true, y_pred = np.array(matrice['points_obsérvés']), np.array(matrice['points_predits'])
MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Prediction Error : %0.2f%%"% MAPE)